In [2]:
# export
from fastai.vision.all import *
import torchvision 
import numpy as np
import torch
import cv2
import os
from modules_th.video_block import *
import random
import time
from pathlib import Path
import os
import pandas as pd

In [3]:
%config Completer.use_jedi = True

In [4]:
torch.cuda.set_device(1)

# Utils functions

In [5]:
#export
def initialize_tensors(bs, l, size):
    fr_ts = torch.zeros(3, size, size, dtype=torch.float)
    snip = Snippet(l, size)
    xb = torch.zeros(bs, l, 3, size, size, dtype=torch.float)
    yb = torch.zeros(bs, dtype=torch.uint8)
    return fr_ts, snip, xb, yb

def iterate_over_videos(df):
    for k in range(len(df)):
        series = df.iloc[k]
        vid_path, start1, end1, start2, end2 = series.file, series.start1, series.end1, series.start2, series.end2       
        vidcap = cv2.VideoCapture(str(vid_path))
        yield vidcap, vid_path, start1, end1, start2, end2
        vidcap.release()

def iterator_over_frames(vidcap, skip=2):
    duration = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width =  int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    l = int(duration//skip)
    frame = np.zeros((height,width,3), dtype=np.uint8)
    for frame_pos in range(0, l*skip, skip):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        res, frame[:] = vidcap.read()
        yield frame_pos, frame        
        
def snippet_is_ready(i, l): return i==l-1
def batch_is_ready(j, bs): return j==bs
        
def get_and_save_embs(xb, yb, model, vid_path, save_path, snip_counter):
    vid_path = Path(vid_path)
    embs = model(xb.permute(0,2,1,3,4).float().cuda())
    #embs = torch.zeros(*xb.shape)
    path = save_path/vid_path.parent.stem/vid_path.stem
    path.mkdir(parents=True, exist_ok=True)
    for i, emb in enumerate(embs):
        torch.save(emb, path/f'{snip_counter:03d}_{yb[i]}.pth')
        snip_counter += 1     
    
class Snippet():
    def __init__(self, l, size):
        self.x = torch.zeros(l, 3, size, size, dtype=float)
        self.l = l
        self.n_saved_frames = 0
        self.n_anomalous_frames = 0
        
    def add_frame(self, fr_ts, lbl_frame):
        self.x[self.n_saved_frames] = fr_ts
        self.n_saved_frames += 1
        self.n_anomalous_frames += lbl_frame
    
    def is_anomaly(self):
        if self.n_anomalous_frames >= self.l*0.5: return 1
        elif self.n_anomalous_frames >= self.l*0.2: return 2
        else: return 0  
    
    def is_ready(self): return self.n_saved_frames==self.l
    
    def restart_counters(self):
        self.n_saved_frames, self.n_anomalous_frames = 0, 0

def fr_is_anomaly(n_fr, start, end):
    if start <= n_fr <= end: return 1
    else: return 0      
    
def add_snip_to_batch(xb, yb, snip, j):
    xb[j], yb[j] = snip.x, snip.is_anomaly()
    snip.restart_counters()

def create_embeddings(df, model, pl, bs, size, l, skip, save_path):    
    for vidcap, vid_path, start1, end1, start2, end2 in iterate_over_videos(df):    
        fr_ts, snip, xb, yb = initialize_tensors(bs, l, size)
        j, snip_counter = 0, 0 # j contera' quanti snippets sono stati gia' messi nel batch 
        for frame_pos, frame in iterator_over_frames(vidcap, skip=skip):
            fr_ts[:], fr_lbl = pl(frame), fr_is_anomaly(frame_pos, start1, end1) or fr_is_anomaly(frame_pos, start2, end2)
            snip.add_frame(fr_ts, fr_lbl)
            if snip.is_ready():
                add_snip_to_batch(xb, yb, snip, j)
                j += 1                
            if batch_is_ready(j,bs): 
                get_and_save_embs(xb, yb, model, vid_path, save_path, snip_counter)
                j, snip_counter = 0, snip_counter + bs
                
        if not batch_is_ready(j,bs): get_and_save_embs(xb[:j+1], yb[:j+1], model, vid_path, save_path, snip_counter)  # if the video didnt fill the last batch, save nevertheless the 
            

# Create Embeddings

## Loading CEL model

In [15]:
model = torch.load('/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/models/model_ucf101_r2p1d50_K_CEL_finetuned_20_5fr.pth')
model = nn.Sequential(model.encoder, model.head[:5])
model = model.eval().cuda()
save_path = Path('/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_CEL_20_5')

##  Loading SCL model

In [12]:
model = torch.load('/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/models/model_ucf101_r2p1d50_K_SCL_finetuned_20_5fr.pth')
model = model.eval().cuda()
save_path = Path('/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5')

In [7]:
model[-1]

Sequential(
  (0): AdaptiveConcatPool3d(
    (ap): AdaptiveAvgPool3d(output_size=1)
    (mp): AdaptiveMaxPool3d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=256, bias=False)
)

## Creating the embeddings 

In [ ]:
df = pd.read_csv('/mnt/data/eugeniomarinelli/UCF_Crimes/Temporal_Anomaly_Annotation.csv', index_col=0)
df = df[df.anomaly!='Normal']
# df = df[2:3]


mean = [0.4345, 0.4051, 0.3775]
std = [0.2768, 0.2713, 0.2737]

bs, size, l, skip = 8, 224, 30, 2
pl = Pipeline([PILImage.create,
               Resize(size, method=ResizeMethod.Pad), 
               ToTensor(), 
               IntToFloatTensor(), 
               Normalize.from_stats(mean,std, cuda=False)])

try: 
    with torch.no_grad():
        create_embeddings(df, model, pl, bs, size, l, skip, save_path)
except: vidcap.release(); raise Exception()

In [11]:
# You fools
#rm_files_at_your_own_risk('/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5')

## L'Opera Incompiuta

In [52]:
path = Path('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/')
for folder0 in path.ls():
    try:
        for folder1 in folder0.ls():
            try: 
                if len(folder1.ls())==0:
                    os.rmdir(folder)
            except: Save csvpass
    except: pass

In [9]:
# export
def rm_files_at_your_own_risk(path,fn=None,ext='.pth'):
    l = get_files(path,extensions=ext)
    l = l.filter(fn) if fn is not None else l
    l.map(os.remove)

## Save .csv file 

In [63]:
import os
df = pd.read_csv('/mnt/data/eugeniomarinelli/UCF_Crimes/Temporal_Anomaly_Annotation.csv')
path = Path('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos')
df.file = L(path/df.anomaly.iloc[i]/ df.file.iloc[i] if df.anomaly[i] != 'Normal' else df.file[i] for i in range(len(df)))
df.head()
#df.to_csv('/mnt/data/eugeniomarinelli/UCF_Crimes/Temporal_Anomaly_Annotation.csv')

,Unnamed: 0,file,anomaly,start1,end1,start2,end2
0,0,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Abuse/Abuse028_x264.mp4,Abuse,165,240,-1,-1
1,1,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Abuse/Abuse030_x264.mp4,Abuse,1275,1360,-1,-1
2,2,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Arrest/Arrest001_x264.mp4,Arrest,1185,1485,-1,-1
3,3,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Arrest/Arrest007_x264.mp4,Arrest,1530,2160,-1,-1
4,4,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Arrest/Arrest024_x264.mp4,Arrest,1005,3105,-1,-1


In [7]:
# Check existence of paths video
all(map(os.path.exists,df.file[df.anomaly!='Normal']))

True

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_losses_tmp.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 05_charades.ipynb.
Converted 07_read_video_function.ipynb.
Converted 08_transformer.ipynb.
Converted 09_lr_schedule.ipynb.
Converted 10_experiments_on_images.ipynb.
Converted 11_create_embeddings.ipynb.
Converted 12_load_r2p1d_model.ipynb.
Converted CALTECH101_MixedLoss.ipynb.
Converted CALTECH101_singleloss.ipynb.
Converted Embeddings_UCF101.ipynb.
Converted UCF101.ipynb.
Converted UCF_Crimes_CEL.ipynb.
Converted UCF_Crimes_SupCon.ipynb.
Converted index.ipynb.


In [27]:
# default_exp create_embeddings

# JUNK

In [31]:
save_path = Path('/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_CEL_20_5')

def rename_embs(snip, vid_path, save_path, snip_counter):
    vid_path = Path(vid_path)
    path = save_path/vid_path.parent.stem/vid_path.stem
    paths_embs = L(path.ls())
    paths_embs.sort()
    paths_embs = paths_embs.filter(lambda x: str(x)[-4:]=='.pth')
    src = paths_embs[snip_counter]
    print(src)
    os.rename(src, path/f'{snip_counter:03d}_{snip.is_anomaly()}.pth')

for vidcap, vid_path, start1, end1, start2, end2 in iterate_over_videos(df):    
    fr_ts, snip, xb, yb = initialize_tensors(bs, l, size)
    snip_counter = 0   
    for frame_pos, frame in iterator_over_frames(vidcap, skip=skip):
        fr_ts[:], fr_lbl = pl(frame), fr_in_anomaly(frame_pos, start1, end1) or fr_in_anomaly(frame_pos, start2, end2)
        snip.add_frame(fr_ts, fr_lbl)
        if snip.is_ready():
            rename_embs(snip, vid_path, save_path, snip_counter)
            snip_counter += 1
            snip.restart_counters()

/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/000_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/001_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/002_2.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/003_1.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/004_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/005_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/006_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/007_0.pth
/mnt/data/eugeniomarinelli/UCF_experiments/embeddings/ucf101_r2p1d50_SCL_20_5/Abuse/Abuse028_x264/008_0.pth
/mnt/data/eugeniomarinelli/U